**Imports :**

In [1]:
import lime
import sklearn.ensemble
import sklearn.linear_model
import sklearn.model_selection
import numpy as np
from sklearn.metrics import r2_score
import lime.lime_tabular
import warnings
from lime import submodular_pick
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import matplotlib
import random
import collections
import math
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree
from six import StringIO
from IPython.display import Image  
import pydotplus
import graphviz

**Creation du jeu de données fictif :**

**Variables d'entrée (numériques ou catégorielles) :**

In [2]:
soilType = ["Hume","Baynes","Antigo", "Jory","Seitz","Miami","Tifton"]
#https://en.wikipedia.org/wiki/Category:Types_of_soil

climate = ["Af", "Am", "BWk", "BSh", "Csc","Cfb", "Dwb", "Dfd", "ET", "EF"]
#https://en.wikipedia.org/wiki/K%C3%B6ppen_climate_classification

# nombre entre 0 et 100%
cultivatedSurface = random.randint(0,100)
equivalenceClasscultivatedSurface =[list(range(0,20)),list(range(20,40)),list(range(40,60)),list(range(60,80)),list(range(80,101))]

# valeur entre 0 et 14
phSoil = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
#https://en.wikipedia.org/wiki/Soil_pH 
equivalenceClassphSoil=[list(range(0,3)),list(range(3,6)),list(range(6,9)),list(range(9,12)),list(range(12,14))]

# les classes d'équivalence sont utilisées lors du calcul de la fonction de couverture

inputFeatures=['soilType','climate','cultivatedSurface','phSoil']

**Variables de sortie (toujours numériques) :**

In [3]:
# valeur en pourcentage
nLeaching = [5,10,15,20,25,30,35,40,45,50,55,65,70,75,80,85,90,95]
#https://en.wikipedia.org/wiki/Leaching_%28agriculture%29
equivalenceClassnLeaching =[list(range(5,25)),list(range(25,50)),list(range(50,75)),list(range(75,100))]

fictionalVariable = [-56,81,78,-1,3,0]
equivalenceClassfictionalVariable = [list(range(-100,-10)),list(range(-10,10)),list(range(10,100))]

outputFeatures=['nLeaching','fictionalVariable']

In [4]:
features=inputFeatures+outputFeatures
numericalFeatures = ['cultivatedSurface','phSoil','nLeaching','fictionalVariable']
categoricalFeatures =['soilType','climate']

**Création jeu de données :**

In [5]:
df = pd.DataFrame(columns = ['soilType','climate','cultivatedSurface','phSoil','nLeaching','fictionalVariable'])

In [6]:
df

,soilType,climate,cultivatedSurface,phSoil,nLeaching,fictionalVariable


**Création de m scénarios :**

In [7]:
m=25

for i in range(m):
    scenario =[]
    soilTypeValue = soilType[random.randint(0,len(soilType)-1)]
    climateValue = climate[random.randint(0,len(climate)-1)]
    cultivatedSurfaceValue = random.randint(0,100)
    phSoilValue = phSoil[random.randint(0,len(phSoil)-1)]
    nLeachingValue = nLeaching[random.randint(0,len(nLeaching)-1)]
    fictionalVariableValue = fictionalVariable[random.randint(0,len(fictionalVariable)-1)]
    scenario.extend([soilTypeValue,climateValue,cultivatedSurfaceValue,phSoilValue,nLeachingValue,fictionalVariableValue])
    df.loc[i] = scenario

In [8]:
df

,soilType,climate,cultivatedSurface,phSoil,nLeaching,fictionalVariable
0,Antigo,Am,19,2,85,78
1,Seitz,BWk,88,8,55,3
2,Seitz,Dfd,65,10,40,-56
3,Seitz,Csc,5,11,80,0
4,Jory,EF,18,2,90,81
5,Miami,Csc,57,6,85,0
6,Tifton,BSh,73,1,90,0
7,Seitz,ET,6,2,70,0
8,Hume,EF,45,2,35,78
9,Antigo,Af,57,14,65,3


**Perturbations des données :**

In [9]:
def createNoneFeatures(numberOfScenarioToEdit,inputt,outputt, noFullSimulatedScenario):
    for i in range(numberOfScenarioToEdit):
        editScenario = random.randint(0,m-1)
        if(inputt):
            index = random.randint(0,len(inputFeatures)-1)
            value=inputFeatures[index]
            df.loc[editScenario,value] = None
        elif(outputt and noFullSimulatedScenario):
            for j in outputFeatures:
                df.loc[editScenario,j] = None
        else :
            index = random.randint(0,len(outputFeatures)-1)
            value=outputFeatures[index]
            df.loc[editScenario,value] = None       

In [10]:
# on modifie 10% des scenarios (en injectant des valeurs None dans des variables)
numberOfScenarioToEdit = int(m * 10/100) 

**Variables de sorties :**

In [11]:
#Variables de sorties entièrement non-simulées
createNoneFeatures(numberOfScenarioToEdit,False,True,True)

In [12]:
#Variables de sortie partiellement non-simulées
createNoneFeatures(numberOfScenarioToEdit,False,True,False)

**Variables d'entrée :**

In [13]:
#Variables d'entrée partiellement non-simulées
createNoneFeatures(numberOfScenarioToEdit,True,False,False)

In [14]:
df

,soilType,climate,cultivatedSurface,phSoil,nLeaching,fictionalVariable
0,Antigo,Am,19,2,85,78
1,Seitz,BWk,88,8,55,3
2,Seitz,None,65,10,40,-56
3,Seitz,Csc,5,11,80,0
4,Jory,EF,18,2,90,81
5,Miami,Csc,57,6,85,0
6,Tifton,BSh,73,1,90,0
7,Seitz,ET,6,2,70,0
8,Hume,None,45,2,35,78
9,Antigo,Af,57,14,65,3


**Fonctions objectives :**

**Fonction de coût :**

In [15]:
# on suppose ici que chaque variable de sortie a un temps de simulation equivalent (1)
def cost_function(scenario_i):
    total =0
    for j in outputFeatures:
        if df.loc[scenario_i,j] is None:
            total=total+1
    return total

**Fonction de couverture :**

In [16]:
def getEquivalenceClass(value,feature):
    equivalenceClass = []
    
    if feature in "equivalenceClasscultivatedSurface" :
        equivalenceClass = equivalenceClasscultivatedSurface

    if feature in "equivalenceClassphSoil" :
        equivalenceClass = equivalenceClassphSoil
    
    if feature in "equivalenceClassnLeaching" :
        equivalenceClass = equivalenceClassnLeaching
        
    if feature in "equivalenceClassfictionalVariable" :
        equivalenceClass = equivalenceClassfictionalVariable
    
    for i in equivalenceClass :
        if value in i :
            return i

In [17]:
def frequency(scenario_i,feature_j):
    presence=0
    value =df.loc[scenario_i,feature_j]
    for scenario_k in range(m):
        if scenario_k!=scenario_i:
            
            if type(value)==str:
                if value == df.loc[scenario_k,feature_j]:
                    presence=presence+1
                    
            if value is None:
                if df.loc[scenario_k,feature_j] is None:
                    presence=presence+1
                    
            if type(value)==int:
                equivalenceClassFeaturefScenarioi = getEquivalenceClass(value,feature_j)
                value_k=df.loc[scenario_k,feature_j]
                equivalenceClassFeaturefScenariok = getEquivalenceClass(value_k,feature_j)
                if collections.Counter(equivalenceClassFeaturefScenarioi) == collections.Counter(equivalenceClassFeaturefScenariok):
                    presence=presence+1
                    
    presence = presence / (len(range(m))-1)
    return presence

In [18]:
def presence(scenario_i,feature_j):
    freq = frequency(scenario_i,feature_j)
    if freq == 0 :
        return freq,0
    else :
        return freq,1

In [19]:
def coverage_function(scenario_i):
    total=0
    for j in features: 
        (freq,pres)=presence(scenario_i,j)
        total= total +(pres/(math.log(freq+2)))
    return total

**Fonction de distance :**

In [20]:
# nous travaillons avec une copie du jeu de données original pour la fonction de distance car celui-ci sera modifié
df_std = df.copy()

**Normalisation en z des données numériques :**

In [21]:
#https://towardsdatascience.com/data-normalization-with-pandas-and-scikit-learn-7c1cc6ed6475
def zNormalisation():
    for j in numericalFeatures:
        df_std[j] = (df_std[j] - df_std[j].mean()) / df_std[j].std()
    for j in numericalFeatures:
        for i in range(m):
            if math.isnan(df_std.loc[i,j]):
                df_std.loc[i,j]=None

In [22]:
zNormalisation()

In [23]:
df_std

,soilType,climate,cultivatedSurface,phSoil,nLeaching,fictionalVariable
0,Antigo,Am,-0.725407,-0.990581,0.998573,1.12251
1,Seitz,BWk,1.51712,0.525615,-0.369038,-0.510145
2,Seitz,None,0.769608,1.03101,-1.05284,-1.7945
3,Seitz,Csc,-1.18041,1.28371,0.770638,-0.575451
4,Jory,EF,-0.757908,-0.990581,1.22651,1.18781
5,Miami,Csc,0.509605,0.0202159,0.998573,-0.575451
6,Tifton,BSh,1.02961,-1.24328,1.22651,-0.575451
7,Seitz,ET,-1.14791,-0.990581,0.314768,-0.575451
8,Hume,None,0.119601,-0.990581,-1.28078,1.12251
9,Antigo,Af,0.509605,2.04181,0.0868325,-0.510145


**Ajout colonnes présences/absences pour variables de sortie**

In [24]:
def addOutputFeaturePresence():
    for j in outputFeatures:
        new_values=[]
        for i in range(m):
            if df_std.loc[i,j] is None:
                new_values.append("False")
            else:
                new_values.append("True")
        df_std[j+"Presence"] = new_values
        categoricalFeatures.append(j+"Presence")       

In [25]:
addOutputFeaturePresence()

In [26]:
df_std

,soilType,climate,cultivatedSurface,phSoil,nLeaching,fictionalVariable,nLeachingPresence,fictionalVariablePresence
0,Antigo,Am,-0.725407,-0.990581,0.998573,1.12251,True,True
1,Seitz,BWk,1.51712,0.525615,-0.369038,-0.510145,True,True
2,Seitz,None,0.769608,1.03101,-1.05284,-1.7945,True,True
3,Seitz,Csc,-1.18041,1.28371,0.770638,-0.575451,True,True
4,Jory,EF,-0.757908,-0.990581,1.22651,1.18781,True,True
5,Miami,Csc,0.509605,0.0202159,0.998573,-0.575451,True,True
6,Tifton,BSh,1.02961,-1.24328,1.22651,-0.575451,True,True
7,Seitz,ET,-1.14791,-0.990581,0.314768,-0.575451,True,True
8,Hume,None,0.119601,-0.990581,-1.28078,1.12251,True,True
9,Antigo,Af,0.509605,2.04181,0.0868325,-0.510145,True,True


**Definition de la distance entre 2 scénarios :**

In [27]:
# on ne compare pas les valeurs NULL entre elles
def distance(scenario_i,scenario_k):
    distance=0
    distanceEuclidienne=0
    distanceHamming=0
    
    denominateurNumerique=0
    denominateurCategoriel=0
    
    for j in numericalFeatures :
        if df_std.loc[scenario_i,j] is None:
            denominateurNumerique=denominateurNumerique+1
        elif df_std.loc[scenario_k,j] is None:
            denominateurNumerique=denominateurNumerique+1       
        else :
            distanceEuclidienne = distanceEuclidienne + (df_std.loc[scenario_i,j] - df_std.loc[scenario_k,j])**2
            
    distanceEuclidienne = math.sqrt(distanceEuclidienne)* (len(numericalFeatures)/(len(df_std.columns)-denominateurNumerique))
         
    for z in categoricalFeatures :

        if df_std.loc[scenario_i,z] is None:
            denominateurCategoriel = denominateurCategoriel+1
                
        elif df_std.loc[scenario_k,z] is None:
            denominateurCategoriel = denominateurCategoriel+1
        
        else:
            if df_std.loc[scenario_i,z]!=df_std.loc[scenario_k,z] :
                distanceHamming=distanceHamming+1
    
    distanceHamming = distanceHamming * (len(categoricalFeatures)/(len(df_std.columns)-denominateurCategoriel))
    
    distance = distanceEuclidienne+distanceHamming
    return distance

**Clustering equitable :**


In [28]:
# fonction qui calcule, pour tous les scénarios de la base de données, leurs plus proche centroid
def closerCentroid(centroid):
    associatedCentroid=[]
    for i in range(m):
        plusProche=centroid[0]
        for j in centroid:
            if distance(i,j)<=distance(i,plusProche):
                plusProche=j
        associatedCentroid.append(plusProche)
    return associatedCentroid      

In [29]:
# fonction qui retourne tous les sous-ensembles possibles de taille "taille" à partir d'un ensemble "liste"
def getAllPossibleSubistOfLength(liste, taille):
    if taille == 0:
        return [[]]
    if len(liste) == 0:
        return []
    l2 = liste[1:]
    subsets = getAllPossibleSubistOfLength(l2, taille-1)
    for s in subsets:
        s.append(liste[0])
    return subsets+ getAllPossibleSubistOfLength(l2, taille)

In [30]:
# fonction qui vérifie si la condition du clustering équitable est vérifiée
def verifyCondition(notcentroid,assignation,sublist):
    for j in notcentroid:
        compteur=0
        
        for i in sublist:
            if  distance(i,j)<=distance(i,assignation[i]):
                compteur=compteur+1
                
        if compteur==len(sublist):
            distanceij=[]
            distanceicentroid=[]
            for i in sublist:
                distanceij.append(distance(i,j))
                distanceicentroid.append(distance(i,assignation[i]))
            return True
        
    return False        

In [31]:
def remove(liste):
    return liste[1:]

In [32]:
# fonction principale qui recherche une solution de taille k vérifiant le clustering équitable en un nombre d'itération fini à partir d'un ensemble "allSublists" qui contient toutes les listes possibles de taille m/k
def fairClustering(k,allSublists,centroids,numMaxIter):
    while numMaxIter>0:
        print("Iteration numero :" +str(numMaxIter))
        centroid=centroids[0]
        notcentroid=[]
        for i in range(m):
            if i not in centroid:
                notcentroid.append(i)
                
        assignation=closerCentroid(centroid)  
        
        for i in allSublists:
            if verifyCondition(notcentroid,assignation,i):
                centroids=remove(centroids)
                numMaxIter=numMaxIter-1
                return fairClustering(k,allSublists,centroids,numMaxIter)


        print("Une solution a ete trouvee")
        print("centroides = " + str(centroid))
        return centroid
    print("Aucune solution trouvée en un certain nombre d iteration")
    return

In [33]:
def launchClustering(k):
    taille = int(m/k)
    x=list(range(m))
    
    allSublists=getAllPossibleSubistOfLength(x,taille)
    allSublists = random.sample(allSublists,len(allSublists))
    
    centroids =getAllPossibleSubistOfLength(x,k)
    centroids = random.sample(centroids,len(centroids))
    
    numMaxIter=500
    
    return fairClustering(k,allSublists,centroids,numMaxIter)

In [34]:
centroid=launchClustering(6)

Iteration numero :500
Iteration numero :499
Iteration numero :498
Iteration numero :497
Iteration numero :496
Iteration numero :495
Iteration numero :494
Iteration numero :493
Iteration numero :492
Iteration numero :491
Iteration numero :490
Iteration numero :489
Iteration numero :488
Iteration numero :487
Iteration numero :486
Iteration numero :485
Iteration numero :484
Iteration numero :483
Iteration numero :482
Iteration numero :481
Iteration numero :480
Iteration numero :479
Iteration numero :478
Iteration numero :477
Iteration numero :476
Iteration numero :475
Iteration numero :474
Iteration numero :473
Iteration numero :472
Iteration numero :471
Iteration numero :470
Iteration numero :469
Iteration numero :468
Iteration numero :467
Iteration numero :466
Iteration numero :465
Iteration numero :464
Iteration numero :463
Iteration numero :462
Iteration numero :461
Iteration numero :460
Une solution a ete trouvee
centroides = [17, 15, 13, 8, 4, 2]


**Si fonction de distance se base sur la moyenne des distance entre un scénario et les centroids :**

In [35]:
"""
def fonction_distance(scenario_i,centroid):
    total=0
    for k in centroid:
        total=total+distance(scenario_i,k)
    total = total/len(centroid)
    return total
"""

'\ndef fonction_distance(scenario_i,centroid):\n    total=0\n    for k in centroid:\n        total=total+distance(scenario_i,k)\n    total = total/len(centroid)\n    return total\n'

**Si fonction de distance se base sur la distance minimum entre un scenario et le plus proche centroid:**

In [36]:
def distance_function(scenario_i,centroid):
    mini = distance(scenario_i,centroid[0])
    for j in range(1,len(centroid)):
        dist=distance(scenario_i,centroid[j])
        if dist < mini:
            mini=dist
    return mini

***Attribution score aux scénarios :***

In [37]:
def score(scenarios):
    res=[]
    for i in scenarios :
        cost=cost_function(i) #minimiser
        coverage = coverage_function(i) #maximiser
        distance = distance_function(i,centroid) #minimiser
        res.append((cost,coverage,distance))
    return res

In [38]:
x=list(range(m))
scores = score(x)

In [39]:
# fonction qui calcule les scores de chaque sous-ensemble appartenant à "possibleSubsets"
def getAllScores(possibleSubsets):
    res=[]
    for i in possibleSubsets:
        scor=[0,0,0]
        for j in i :
            scor = tuple(map(sum,zip(scor,scores[j])))
        test=[i]
        test.append(scor)
        res.append(test)
    return res

In [40]:
# fonction qui retourne le meilleur (en terme de dominance Pareto) sous-ensemble de la liste "scores" 
def getBestParetoSubset(scores):
        bestlist =scores[0][0]
        bestscore= scores[0][1]
        for i in range(1,len(scores)):
            if  scores[i][1][0] < bestscore[0]:
                if scores[i][1][1] >= bestscore[1]:
                    if scores[i][1][2] <= bestscore[2]:
                        bestlist = scores[i][0]
                        bestscore = scores[i][1]
                        
            elif  scores[i][1][1] > bestscore[1]:
                if scores[i][1][0] <= bestscore[0]:
                    if scores[i][1][2] <= bestscore[2]:
                        bestlist = scores[i][0]
                        bestscore = scores[i][1]
            
            elif  scores[i][1][2] < bestscore[2]:
                if scores[i][1][0] <= bestscore[0]:
                    if scores[i][1][1] >= bestscore[1]:
                        bestlist = scores[i][0]
                        bestscore = scores[i][1]

        return bestlist,bestscore

In [41]:
# fonction qui retourne les meilleurs ensembles de la taille "taille" à la taille "maxi" en terme de Pareto sur base des trois fonctions objectives
def subsetmin(B,taille,maxi,data):
    if maxi == 1 :
        sub_pick=getAllPossibleSubistOfLength(data,taille)
        allScores = getAllScores(sub_pick)
        a=getBestParetoSubset(allScores)
        B.append(a)
        return B
    
    elif taille == maxi :
        sub_pick=getAllPossibleSubistOfLength(data,taille)
        allScores = getAllScores(sub_pick)
        a=getBestParetoSubset(allScores)
        B.append(a)
        return B
    
    else: 
        sub_pick=getAllPossibleSubistOfLength(data,taille)
        allScores = getAllScores(sub_pick)
        a=getBestParetoSubset(allScores)
        B.append(a)
        taille=taille+1
        return subsetmin(B,taille,maxi,data)

In [42]:
subset=subsetmin([],1,25,x)

In [43]:
for i in subset:
    print(str(i[0]) + "" + str(i[1]))

[2](0, 8.083085439748809, 0.0)
[8, 2](0, 15.915731560445614, 0.0)
[15, 8, 2](0, 23.553225484620576, 0.0)
[15, 13, 8, 2](0, 31.1640721225833, 0.0)
[15, 13, 8, 2, 1](0, 38.73309253684358, 0.8560589531920317)
[15, 13, 8, 4, 2, 1](0, 46.260235461104486, 0.8560589531920317)
[18, 15, 13, 8, 4, 2, 1](0, 53.94049282124, 2.035418907013973)
[18, 15, 13, 9, 8, 4, 2, 1](0, 61.66887185466688, 3.6115535107112526)
[19, 18, 15, 13, 8, 4, 2, 1, 0](0, 68.99419175811317, 4.523790485391753)
[19, 18, 15, 13, 9, 8, 4, 2, 1, 0](0, 76.72257079154006, 6.099925089089032)
[23, 19, 18, 15, 13, 9, 8, 4, 2, 1, 0](0, 84.280195669599, 7.415034523571375)
[23, 19, 18, 15, 13, 9, 8, 4, 3, 2, 1, 0](0, 91.83485027980957, 8.88792236906166)
[23, 19, 18, 17, 15, 13, 9, 8, 4, 3, 2, 1, 0](1, 99.7610061295205, 8.88792236906166)
[23, 19, 18, 17, 15, 13, 9, 8, 5, 4, 3, 2, 1, 0](1, 107.37185276748322, 10.417971570690327)
[23, 19, 18, 17, 15, 13, 12, 9, 8, 5, 4, 3, 2, 1, 0](2, 115.07998250342516, 11.870256446302676)
[23, 19, 18, 17

In [45]:
#minimum 25% du set de depart
minimalSizeTrainingSet = int(m * 27/100) -1
solution=getBestParetoSubset(subset[minimalSizeTrainingSet:])
print(solution)

([15, 13, 8, 4, 2, 1], (0, 46.260235461104486, 0.8560589531920317))


In [47]:
trainingSet=solution[0]
trainingSet.reverse()
print(trainingSet)

[1, 2, 4, 8, 13, 15]
